In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install pycaret
from pycaret.classification import *
#from pycaret.regression import *

# Importing the data - only use *.MLReady.csv files!!!

In [2]:
df=pd.read_csv("../../data/FM_FULL_EPOCH2_MLReady.csv")
df.head()

,Unnamed: 0,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,...,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,0,2,5.125,348000,87,1,50,689,2,51,...,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975,2009,2
1,1,3,4.625,195000,52,2,54,703,1,82,...,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441,2009,2
2,2,2,4.875,342000,80,1,54,746,1,981,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2
3,3,1,5.375,93000,70,1,50,780,1,496,...,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758,2009,2
4,4,1,4.875,182000,76,2,22,802,1,18,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 32 columns):
Unnamed: 0            119696 non-null int64
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int64
numBorrowers          119696 non-null int64
origDebtIncRatio      119696 non-null int64
borrCreditScore       119696 non-null int64
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int64
bestCreditScore       119696 non-null int64
worstCreditScore      119696 non-null int64
avgCreditScore        119696 non-null int64
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null int64
fmacRateMax           119696 non-null float64
fma

## Remove columns not involved in 'Foreclosure or Not' query

In [4]:
df.drop(['Unnamed: 0'],1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int64
numBorrowers          119696 non-null int64
origDebtIncRatio      119696 non-null int64
borrCreditScore       119696 non-null int64
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int64
bestCreditScore       119696 non-null int64
worstCreditScore      119696 non-null int64
avgCreditScore        119696 non-null int64
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null int64
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64
f

# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [6]:
df.zeroBalCode.unique()

array([1, 0], dtype=int64)

In [7]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,16,6,2,15],1)

In [8]:
df.zeroBalCode.unique()

array([0], dtype=int64)

In [9]:
#Checking datatypes of individual feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int64
numBorrowers          119696 non-null int64
origDebtIncRatio      119696 non-null int64
borrCreditScore       119696 non-null int64
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int64
bestCreditScore       119696 non-null int64
worstCreditScore      119696 non-null int64
avgCreditScore        119696 non-null int64
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null int64
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64
f

# Review values for dates and slice (if needed) the timeframe you want

In [10]:
df.origYear.unique()

array([2009, 2010, 2011, 2012, 2013], dtype=int64)

In [11]:
df['origMonth'].unique()

array([ 2,  3,  5,  4,  6,  7,  8,  9, 10, 11, 12,  1], dtype=int64)

# Coerce the column dtype (int in this case)

In [12]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'object'})
#,'zeroBalCode':'int'

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int32
numBorrowers          119696 non-null int32
origDebtIncRatio      119696 non-null int32
borrCreditScore       119696 non-null int32
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int32
bestCreditScore       119696 non-null int32
worstCreditScore      119696 non-null int32
avgCreditScore        119696 non-null int32
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null object
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64


In [14]:
#df.to_csv('D:\FM_FULL_EPOCH2_MLReady.csv')

# Review metrics last time before run (to have record of starting values)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int32
numBorrowers          119696 non-null int32
origDebtIncRatio      119696 non-null int32
borrCreditScore       119696 non-null int32
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int32
bestCreditScore       119696 non-null int32
worstCreditScore      119696 non-null int32
avgCreditScore        119696 non-null int32
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null object
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64


In [16]:
df.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,...,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,2,5.125,348000,87,1,50,689,2,51,25.0,...,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975,2009,2
1,3,4.625,195000,52,2,54,703,1,82,0.0,...,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441,2009,2
2,2,4.875,342000,80,1,54,746,1,981,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2
3,1,5.375,93000,70,1,50,780,1,496,0.0,...,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758,2009,2
4,1,4.875,182000,76,2,22,802,1,18,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2


In [17]:
model_setup = setup(df, target = 'zeroBalCode', pca = True, pca_components = 5, ignore_low_variance = True, 
            categorical_features = ['numBorrowers', 'mortInsType', 'zipCode', 'bankNumber',
                                    'stateNumber'],
            numeric_features = ['origLTV', 'origDebtIncRatio', 'borrCreditScore', 'bestCreditScore', 'worstCreditScore',
                               'avgCreditScore'])

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:52:45
Status,. . . . . . . . . . . . . . . . . .,Fitting Model on 10% sample
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
origChannel,Categorical
origIntRate,Numeric
origUPB,Numeric
origLTV,Numeric
numBorrowers,Categorical
origDebtIncRatio,Numeric
borrCreditScore,Numeric
loanPurp,Categorical
zipCode,Categorical
pMIperct,Numeric


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

# Pycaret - Setup with categorical definition

### Use feature engineering, polynomial features, trig feats, PCA

### Use feature engineering, polynomial features, trig feats

## Compare Models with blacklist exclusions

In [ ]:
model_results=compare_models()
model_results

## Create models - store in variable

In [ ]:
cat = create_model('catboost', fold = 10)

## Tune Models

In [ ]:
tuned_cat = tune_model('catboost')

## Ensemble models

In [ ]:
cat_bagged = ensemble_model(cat)

## Evaluate Models

In [ ]:
interpret_model(estimator=cat, plot='correlation')

In [ ]:
interpret_model(estimator=cat, plot='summary')